Do custom install of `sage-importance`

```bash
git clone https://github.com/karelze/sage.git
cd sage
pip install .
```

In [ ]:
import os
import sys
import pickle
from pathlib import Path

from catboost import CatBoostClassifier

import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.ensemble import HistGradientBoostingClassifier
from torch import nn

sys.path.append("..")
from otc.models.classical_classifier import ClassicalClassifier

from sage import GroupedMarginalImputer, PermutationEstimator

from otc.features.build_features import (
    features_categorical,
    features_classical,
    features_classical_size,
    features_ml,
)

import wandb
from tqdm.auto import tqdm

In [ ]:
SEED = 42

# set globally here
EXCHANGE = "ise"  
STRATEGY = "supervised"  
SUBSET = "test"  

# Change depending on model!
FEATURES = features_classical_size

In [ ]:
# set project name. Required to access files and artefacts
os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

In [ ]:
# see https://wandb.ai/fbv/thesis/runs/kwlaw02g/overview?workspace=user-karelze
dataset = f"fbv/thesis/{EXCHANGE}_{STRATEGY}_none:latest"
run = wandb.init(project="thesis", entity="fbv")

artifact = run.use_artifact(dataset)
data_dir = artifact.download()

## Data Preparation 🌊

In [ ]:
run = wandb.init(project="thesis", entity="fbv")

dataset = f"fbv/thesis/{EXCHANGE}_{STRATEGY}_none:latest"

artifact = run.use_artifact(dataset)
data_dir = artifact.download()

columns = [
    *FEATURES,
    "buy_sell",
]

data = pd.read_parquet(Path(data_dir, "train_set.parquet"), engine="fastparquet", columns=columns)

y_train_none = data["buy_sell"]
X_train_none = data.drop(columns="buy_sell")

data = pd.read_parquet(Path(data_dir, "test_set.parquet"), engine="fastparquet", columns=columns)

y_test_none = data["buy_sell"]
X_test_none = data.drop(columns="buy_sell")


dataset = f"fbv/thesis/{EXCHANGE}_{STRATEGY}_log_standardized_clipped:latest"

artifact = run.use_artifact(dataset)
data_dir = artifact.download()

data = pd.read_parquet(Path(data_dir, "train_set.parquet"), engine="fastparquet", columns=columns)

y_train_processed = data["buy_sell"]
X_train_processed = data.drop(columns="buy_sell")

data = pd.read_parquet(Path(data_dir, "test_set.parquet"), engine="fastparquet", columns=columns)

y_test_processed = data["buy_sell"]
X_test_processed = data.drop(columns="buy_sell")

feature_names = X_train_none.columns

## Sage Values🌵

In [ ]:
# Define feature groups (disjoint)
feature_groups = group_names = {
    'chg_all_lead (grouped)': ['price_all_lead', 'chg_all_lead'],
    'chg_all_lag (grouped)': ['price_all_lag', 'chg_ex_lag'],
    'chg_ex_lead (grouped)': ['price_ex_lead', 'chg_ex_lead', 'chg_all_lag'],
    'chg_ex_lag (grouped)': ['price_ex_lag'],
    'size_ex (grouped)': [ 'bid_ask_size_ratio_ex', 'rel_bid_size_ex',  'rel_ask_size_ex', 'bid_size_ex', 'ask_size_ex','depth_ex'],
    'quote_best (grouped)': ['BEST_ASK', 'BEST_BID', 'prox_best'],
    'quote_ex (grouped)': ['bid_ex', 'ask_ex','prox_ex' ],
    'TRADE_PRICE': ['TRADE_PRICE'],
    'TRADE_SIZE': ['TRADE_SIZE']    
}
group_names = [group for group in feature_groups]
for col in feature_names:
    if np.all([col not in group[1] for group in feature_groups.items()]):
        group_names.append(col)

# Group indices
groups = []
for _, group in feature_groups.items():
    ind_list = []
    for feature in group:
        ind_list.append(feature_names.tolist().index(feature))
    groups.append(ind_list)


### Classical Classifier🏦

In [ ]:
clf = ClassicalClassifier(layers=[("trade_size", "ex"), ("rev_lr", "best")], 
                                  random_state=SEED, strategy="random")

clf.fit(X=X_train_none.head(5), y=y_train_none.head(5))

imputer = GroupedMarginalImputer(clf, X_test_none.head(1024).values, groups)
estimator = PermutationEstimator(imputer, "cross entropy")
sage_values = estimator(X_test_none.head(1024).values, y_test_none.head(1024).values)

In [ ]:
sage_values.plot(group_names, title=None)

### Gradient Boosting 🐈

In [ ]:
# load model by identifier from wandb
model = "17malsep_CatBoostClassifier_default.cbm:v7"
model_name = model.split("/")[-1].split(":")[0]

artifact = run.use_artifact(model)
model_dir = artifact.download()

clf = CatBoostClassifier()
clf.load_model(fname=Path(model_dir, model_name))

In [ ]:
imputer = GroupedMarginalImputer(clf, X_test_processed.head(128).values, groups)
estimator = PermutationEstimator(imputer, "cross entropy")
sage_values = estimator(X_test_processed.head(128).values, y_test_processed.head(128).values)

In [ ]:
sage_values.plot(group_names, title=None)

### Transformer Classifier 🤖

In [ ]:
model = "2rq3hrkw_TransformerClassifier_default.pkl:latest"
model_name = model.split("/")[-1].split(":")[0]

artifact = run.use_artifact(model)
model_dir = artifact.download()
    
with open(Path(model_dir, model_name), 'rb') as f:
    clf = pickle.load(f)

In [ ]:
imputer = GroupedMarginalImputer(clf, X_test_processed.head(1024).values, groups)
estimator = PermutationEstimator(imputer, "cross entropy")
sage_values = estimator(X_test_processed.head(1024).values, y_test_processed.head(1024).values)

In [ ]:
sage_values.plot(group_names, title=None)

In [ ]:
model = "i3pvza1q_TransformerClassifier_default.pkl:latest"
model_name = model.split("/")[-1].split(":")[0]

artifact = run.use_artifact(model)
model_dir = artifact.download()
    
with open(Path(model_dir, model_name), 'rb') as f:
    clf = pickle.load(f)

## Attention Maps for Transformers

We calculate the average attention map from all transformer blocks, as done in the Gorishniy paper (see [here](https://github.com/Yura52/tabular-dl-revisiting-models/issues/2)). This is different from the Borisov paper (see [here](https://github.com/kathrinse/TabSurvey/blob/main/models/basemodel_torch.py)).

In [ ]:
class SaveAttentionMaps:
    """
    Hook for attention maps.

    Inspired by:
    https://github.com/Yura52/tabular-dl-revisiting-models/issues/2#issuecomment-1068123629
    """

    def __init__(self):
        self.attention_maps = []

    def __call__(self, _, __, output):
        self.attention_maps.append(output[1]["attention_probs"])

In [ ]:
import torch
from torch import nn

from otc.models.activation import ReGLU
from otc.models.fttransformer import (
    FeatureTokenizer,
    FTTransformer,
    Transformer,
)

num_features_cont = 5
num_features_cat = 1
cat_cardinalities = [2]
batch_size = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x_cat = torch.randint(0, 1, (batch_size, num_features_cat)).to(device)
x_cont = torch.randn(batch_size, num_features_cont).float().to(device)
expected_outputs = torch.randint(0, 1, (batch_size, 1)).float().to(device)
expected_outputs.requires_grad_(True)

params_feature_tokenizer = {
    "num_continous": num_features_cont,
    "cat_cardinalities": cat_cardinalities,
    "d_token": 96,
}
feature_tokenizer = FeatureTokenizer(**params_feature_tokenizer)
params_transformer = {
    "d_token": 96,
    "n_blocks": 3,
    "attention_n_heads": 8,
    "attention_initialization": "kaiming",
    "ffn_activation": ReGLU,
    "attention_normalization": nn.LayerNorm,
    "ffn_normalization": nn.LayerNorm,
    "ffn_dropout": 0.1,
    "ffn_d_hidden": 96 * 2,
    "attention_dropout": 0.1,
    "residual_dropout": 0.1,
    "prenormalization": True,
    "first_prenormalization": False,
    "last_layer_query_idx": None,
    "n_tokens": None,
    "kv_compression_ratio": None,
    "kv_compression_sharing": None,
    "head_activation": nn.ReLU,
    "head_normalization": nn.LayerNorm,
    "d_out": 1,
}

transformer = Transformer(**params_transformer)

model = FTTransformer(feature_tokenizer, transformer).to(device)

In [ ]:
output = model(x_cat, x_cont)
one_hot = output.sum()

In [ ]:
model.zero_grad()

In [ ]:
# https://github.com/hila-chefer/Transformer-MM-Explainability/blob/main/lxmert/lxmert/src/ExplanationGenerator.py#L26
# https://colab.research.google.com/github/hila-chefer/Transformer-MM-Explainability/blob/main/CLIP_explainability.ipynb#scrollTo=fWKGyu2YAeSV

# cat + cont + [CLS]
num_tokens = attn_blocks[0].shape[-1]
# residual connection. Repeat along batch dimension
res = torch.eye(num_tokens, num_tokens, dtype=attn_blocks[0].dtype).to(device)
res = res.unsqueeze(0).expand(batch_size, num_tokens, num_tokens)
for i, attn_probs in enumerate(hook.attention_maps):

    # TODO: Fix gradient calculation
    grad = torch.autograd.grad(one_hot, [attn_probs], retain_graph=True)[0].detach()

    # get attention maps
    cam = blk.detach()
    cam = cam.reshape(-1, cam.shape[-1], cam.shape[-1])
    grad = grad.reshape(-1, grad.shape[-1], grad.shape[-1])
    cam = grad * cam
    cam = cam.reshape(batch_size, -1, cam.shape[-1], cam.shape[-1])
    cam = cam.clamp(min=0).mean(dim=1)
    res = res + torch.bmm(cam, res)

relevancy = res
# disregard the first token, which is the [CLS] token
relevancy[:,0,0] = 0